# Analyzing Livestream Data from Twitch 

In order to connect to Twitch chat, one should use Python Sockets to connect via IRC. To stream messages from the Twitch IRC, you need to get a token for authentication. To do that, you need a Twitch account. I've created a development team account. Now, you need to go to the following [link]( https://twitchapps.com/tmi/) while signed in to obtain an Auth token for your Twitch account. You click authorize and keep that token handy for later. 

Below, we define a few constants for obtaining Twitch data. Channel responds to the streamer's name and can be the name of any channel one is interested in. Currently, this is set up for one chat stream at a time. 

In [ ]:
# Important Constants for Connection to Twitch Channel Feed
server = 'irc.chat.twitch.tv'
port = 6667
nickname = 'acn_development_team' # '<YOUR_USERNAME>'
token = 
channel = '#giantbomb' # The hashtag before the username here is SUPER important. 

# Connecting to Twitch using Sockets

To establish a connection to Twitch IRC, we'll use Python's socket library. So we first instantiate a socket:

In [ ]:
import socket
sock = socket.socket()
sock.settimeout(120)

Next, we need to connect the socket to Twitch by calling connect() with the server and port previously defined. 

In [ ]:
sock.connect((server,port))

Once connected, we need to send our token and nickname for authentication as well as the channel we care about. These are sent via encoded strings, so we need to send them in this very specific format: 

In [ ]:
# where 'utf-8' encoding encodes the string into bytes which allows it to be sent over the socket. 
sock.send(f"PASS {token}\n".encode('utf-8')) # carries token 
sock.send(f"NICK {nickname}\n".encode('utf-8')) # carries nickname
sock.send(f"JOIN {channel}\n".encode('utf-8')) # carries channel 

In [ ]:
# Recieving Channel Messages 
# Example: Getting a single response. 
# This is commented out because it's just for testing out one example. 

# resp = sock.recv(2048).decode('utf-8') #2048 is the buffer size in bytes, amount of data to recieve. 
# resp


# sock.close() # Use this to open/close the socket. 

In [ ]:
# Currently, the socket gets responses but we need to check for new messages and log messages as they come in. 
# The proper way to do this is to set up a logger that writes messages to a file and a loop that will check for new messages as the socket's open. 

import logging

logging.basicConfig(level=logging.DEBUG,
                    format='%(asctime)s — %(message)s',
                    datefmt='%Y-%m-%d_%H:%M:%S',
                    handlers=[logging.FileHandler('chat2.log', encoding='utf-8')])

# Debug means all levels of logging can be written to the file. 
# Format is how we want the line to look, which will be time/message. Date format is how the time of the format is recorded. 
# FileHandler to handlers is passed which creates a file in the directory and logs info to chat.log. 

In [ ]:
# logging.info(resp) # We can open our repository and notice chat.log is updated now! 

In [ ]:
# Continuously checking for new messages in a loop. When connected to IRC, we want to make sure to send "PONG" if server sends "PING."
# Also want to parse emojis so they can be written to a file. There's an emoji library that maps emoji to word meaning. 

from emoji import demojize

while True:
    resp = sock.recv(2048).decode('utf-8')

    if resp.startswith('PING'):
        sock.send("PONG\n".encode('utf-8'))
    
    elif len(resp) > 0:
        logging.info(demojize(resp))
        
# This will keep running until you stop it. 
# To see the messages in real-time open a new terminal, navigate to the log's location, and run tail -f chat.log.

In [ ]:
sock.close() # Use this to open/close the socket. 

# Parsing the Chat Logs from Twitch
In this case, we want to parse the chat log into a Pandas DataFrame to prepare for analysis. This means that we want to have the username, message, date, and time information (but really I care only about the message). Because we need to get data from each line, we need to be strategic about how we read info.

In [ ]:
import pandas as pd
from datetime import datetime
import re

def get_chat_dataframe(file):
    data = []

    with open(file, 'r', encoding='utf-8') as f:
        lines = f.read().split('\n')
        
        for line in lines:
            try:
                time_logged = line.split('—')[0].strip()
                time_logged = datetime.strptime(time_logged, '%Y-%m-%d_%H:%M:%S')

                username_message = line.split('—')[1:]
                username_message = '—'.join(username_message).strip()

                username, channel, message = re.search(
                    ':(.*)\!.*@.*\.tmi\.twitch\.tv PRIVMSG #(.*) :(.*)', username_message
                ).groups()

                d = {
                    'dt': time_logged,
                    'channel': channel,
                    'username': username,
                    'message': message
                }

                data.append(d)
            
            except Exception:
                pass
            
    return pd.DataFrame().from_records(data)

In [ ]:
df = get_chat_dataframe('chat.log')

df.set_index('dt', inplace=True)

print(df.shape)

df.head()


In [ ]:
# Now, employing my own custom data processing. I'm personally only interested in the messages feature! 
df['message'].head()

# Cleaning Dataset, Visualizing Datset, and Building LDA Model 

In [ ]:
# Import modules

import numpy as np
import pandas as pd
import re, nltk, spacy, gensim

#TextHero
# !pip install texthero -U   
import texthero as hero

In [ ]:
#Pass a custom pipeline as argument to clean

from texthero import preprocessing
# https://pypi.org/project/texthero/
# Texthero is a really good library built off of Pandas and it allows for those with minimal knowledge of the NLP space to utilize powerful tools. 

custom_pipeline = [preprocessing.lowercase, # much easier than previous solution which was to df.remove. 
                   preprocessing.remove_punctuation,
                   preprocessing.remove_urls]
data = hero.clean(df['message'])

In [ ]:
# Custom STOPWORDS removal
from texthero import stopwords
default_stopwords = stopwords.DEFAULT
custom_stopwords = default_stopwords.union(set(["twitch","make", "use", "thank", "content", "good", "use",
                                                "think", "need", "harrisheller", "like", "stream",
                                               "kekw","catjam","tim","timthetatman","tatkevinh",
                                               "wipz","docspin","pog","tatlove","lol","lul","omegalul",
                                               "biblethump","clap","tathypers","pepeja","kappa","tattopd","ppsmoke",
                                               "pepelaugh","gopackgo","gachihyper","tatkevinh", "wipz",
                                               "pausechamp","yep","lmao","jack","lulw","monkaw","kreygasm",
                                               "pepega","peped","foxsalt","pogchamp","xqcn","get","back",
                                               "tattuff","tatfat","tatpumpkin","lmao","sadge","sippy",
                                               "pogu","poggers","consolecd","widepeepohappy","pogu","tategg2",
                                               "modcheck","timmy","tathmm","tats","got","com","babyrage",
                                               "xqcp","tatw","pokiw","know", "thats","pepocd","tatafk",
                                               "4weird","tatkkevin", "tatblanket","tatglam","tategg1","wutface",
                                               "blobdance", "kapp","tatbruh","kappapride","facebaby","xqc",
                                               "xqcm","bora","hyperclap","tatlit","5head","gachibass", "go", "ur",
                                                "yes","going","would","im","oh","dez","taty","tk","u","sg", "dont",
                                                "hey","hf","look"
                                               ])) ## Add as per requirement
# data = hero.remove_stopwords(data, default_stopwords)
data = hero.remove_stopwords(data, custom_stopwords)

In [ ]:
hero.visualization.wordcloud(data, font_path = None, width = 400, height = 200, max_words=200, 
                             mask=None, contour_width=0, 
                             contour_color='PAPAYAWHIP', background_color='WHITE', 
                             relative_scaling='auto', colormap=None, return_figure=False)

In [ ]:
# Tokenize

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

In [ ]:
# Lemmatize
# Lemmatization usually refers to doing things properly with the use of a vocabulary and morphological analysis of words, 
# normally aiming to remove inflectional endings only and to return the base or dictionary form of a word, which is known as the lemma.
# tl;dr put the verbs in their "stem" form. 

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)

nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only Noun, Adjective, Verb, Adverb
data_lemmatized = lemmatization(data_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:50])

In [ ]:
# Word-Document Matrix
# This converts a collection of text documents to a matrix of token counts. 
# A document-term matrix or term-document matrix is a mathematical matrix that describes the frequency of terms that occur in a collection of documents. 
# In a document-term matrix, rows correspond to documents in the collection and columns correspond to terms. 

vectorizer = CountVectorizer(analyzer='word',       
                             min_df=10,                        # minimum number occurences of a word required
                             stop_words='english',             # remove stop words
                             lowercase=True,                   # convert all words to lowercase
                             token_pattern='[a-zA-Z0-9]{3,}')  # num of characters > 3
                            

data_vectorized = vectorizer.fit_transform(data_lemmatized)

In [ ]:
# Sklearn
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint

# Plotting tools
# !pip install -U pyLDAvis
import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Build LDA model with Sklearn

lda_model = LatentDirichletAllocation(n_components=15,               # Number of topics
                                      max_iter=10,                   # Max learning iterations
                                      learning_method='online',   
                                      random_state=100,              # Random state
                                      batch_size=128)                # n docs in each learning iter
                                            
                                      
lda_output = lda_model.fit_transform(data_vectorized)

print(lda_model)  # Model attributes

In [ ]:
#Evaluate model quality

# Log Likelihood: Higher the better
print("Log Likelihood: ", lda_model.score(data_vectorized))

# Perplexity: Lower the better. Perplexity = exp((-1) * log-likelihood per word)
print("Perplexity: ", lda_model.perplexity(data_vectorized))


In [ ]:
# Grid Search for Best LDA Model
# Grid search is a tuning technique that attempts to compute the optimum values of hyperparameters. 
# It is an exhaustive search that is performed on a the specific parameter values of a model. The model is also known as an estimator.
# This is computationally expensive and usually takes time... 

from sklearn.model_selection import GridSearchCV

# Define Search Param
params = {'n_components': [10, 15, 20, 25, 30], 'learning_decay': [.5, .7, .9]}

# Model
lda = LatentDirichletAllocation()

# Grid Search
model = GridSearchCV(lda, param_grid=params)

# Perform Grid Search
model.fit(data_vectorized)

In [ ]:
# Best Model
best_lda_model = model.best_estimator_

# Model Parameters
print("Best Model's Params: ", model.best_params_)

# Log Likelihood Score
print("Best Log Likelihood Score: ", model.best_score_)

# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(data_vectorized))

In [ ]:
#Visualize with pyLDAvis

pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(best_lda_model, data_vectorized, vectorizer, mds='tsne')
panel

In [ ]:
# Reflections:
# This exercise shows how you can analyze a massive corpus of text and split data into "topics" which contain certain words that appear 
# frequently amongst those topics. In this case, we have 10 topics and 10 words that reflect those topics. What this data can tell us is 
# how important and how frequent certain words are in the corpus - what themes/etc. may emerge. 

# Show top n keywords for each topic

def show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=20):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords

topic_keywords = show_topics(vectorizer=vectorizer, lda_model=best_lda_model, n_words=10)        

# Topic - Keywords Dataframe
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords

In [ ]:
# Now, focusing on Semantic Text Similarity.
# First, we're adding our import statements. 
import pandas as pd
import numpy as np
import spacy
import os 
import re
import operator
import pickle
import nltk 
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# Now, doing Word Tokenization
# Tokenization is hwen each entry in the data is broken down into a set of words.
import nltk
nltk.download('punkt')

df['Word tokenize']= [word_tokenize(entry) for entry in df.message]

In [ ]:
# WordNetLemmatizer requires Pos tags to understand if the word is noun or verb or adjective etc. By default it is set to Noun
def wordLemmatizer(data):
    tag_map = defaultdict(lambda : wn.NOUN)
    tag_map['J'] = wn.ADJ
    tag_map['V'] = wn.VERB
    tag_map['R'] = wn.ADV
    file_clean_k =pd.DataFrame()
    for index,entry in enumerate(data):
        
        # Declaring Empty List to store the words that follow the rules for this step
        Final_words = []
        # Initializing WordNetLemmatizer()
        word_Lemmatized = WordNetLemmatizer()
        # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
        for word, tag in pos_tag(entry):
            # Below condition is to check for Stop words and consider only alphabets - I turned this part off for now. 
#             if len(word)>1 and word not in stopwords.words('english') and word.isalpha():
                word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
                Final_words.append(word_Final)
            # The final processed set of words for each iteration will be stored in 'text_final'
                file_clean_k.loc[index,'Keyword_final'] = str(Final_words)
                file_clean_k.loc[index,'Keyword_final'] = str(Final_words)
                #file_clean_k=file_clean_k.replace(to_replace ="\[.", value = '', regex = True)
                #file_clean_k=file_clean_k.replace(to_replace ="'", value = '', regex = True)
                #file_clean_k=file_clean_k.replace(to_replace =" ", value = '', regex = True)
                #file_clean_k=file_clean_k.replace(to_replace ='\]', value = '', regex = True)
    return file_clean_k

In [ ]:
# # My version of lemmatization based on the Coursera code/work I've done previously... 

# # Lemmatization usually refers to doing things properly with the use of a vocabulary and morphological analysis of words, 
# # normally aiming to remove inflectional endings only and to return the base or dictionary form of a word, which is known as the lemma.
# # tl;dr put the verbs in their "stem" form. 

# def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
#     """https://spacy.io/api/annotation"""
#     texts_out = []
#     for sent in texts:
#         doc = nlp(" ".join(sent)) 
#         texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
#     return texts_out

# # Initialize spacy 'en' model, keeping only tagger component (for efficiency)

# nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])


In [ ]:
df.shape

In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

# df_clean = wordLemmatizer(df['Word tokenize'][0:10]) 
# FOR THE WHOLE DATASET: 
df_clean = wordLemmatizer(df['Word tokenize'])
df_clean.shape

In [ ]:
df_clean=df_clean.replace(to_replace ="\[.", value = '', regex = True)
df_clean=df_clean.replace(to_replace ="'", value = '', regex = True)
df_clean=df_clean.replace(to_replace =" ", value = '', regex = True)
df_clean=df_clean.replace(to_replace ='\]', value = '', regex = True)
df_clean

In [ ]:
# Do lemmatization keeping only Noun, Adjective, Verb, Adverb
# This is using my version. 
# df_clean = lemmatization(df['Word tokenize'], allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
# print(df_clean)

In [ ]:
df_clean.size

In [ ]:
#Add Lemmatized words to the dataframe
## Insert New column in df to stored the Clean Keyword
df.insert(loc=4, column='Clean_Keyword', value=df_clean['Keyword_final'].tolist())

In [ ]:
df

In [ ]:
# df = df.drop(['Word tokenize','Clean_Keyword'],axis=1)

In [ ]:
# df.to_csv("df_twitch.csv", index=False, header=True)

In [ ]:
df.Clean_Keyword[0]

In [ ]:
!pip install --upgrade tensorflow-gpu
#Install TF-Hub.
!pip install tensorflow-hub
!pip install seaborn

In [ ]:
# Yay more imports
import pandas as pd
import numpy as np
import re, string
import os 
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import linear_kernel

In [ ]:
print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")

Now, using TF-IDF to create a document search tool. TF-IDF is the process of calculating the weight of each word (signifying the importance of the word in the corpus/document). The algorithm is used mainly for retrieving information and text mining.

TF (Term Frequency) is how many times a word appears in a document divided by the number of the words in the document.

IDF (Inverse Data Frequency) is the log of the number of documents divided by the number of documents with the word W that we're interested in.

TF-IDF is just these two numbers multiplied together - sklearn implements this feature for you in their library.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import operator

## Create Vocabulary
vocabulary = set()

for doc in df.Clean_Keyword:
    vocabulary.update(doc.split(','))

vocabulary = list(vocabulary)

# Intializating the tfIdf model
tfidf = TfidfVectorizer(vocabulary=vocabulary,dtype=np.float32)

# Fit the TfIdf model
tfidf.fit(df.Clean_Keyword)

# Transform the TfIdf model
tfidf_tran=tfidf.transform(df.Clean_Keyword)

In [ ]:
vocabulary[0:10] # for example

In [ ]:
vocabulary = list(filter(None, vocabulary)) # just removing the empty '' in the list! 

In [ ]:
vocabulary[0:10]

In [ ]:
# Saving the trained TF-ID Model
# I uploaded this to Colab and then ran this. 
with open('/Users/ronak.k.bhatia/Desktop/Data_STA/tfid.pkl','wb') as handle:
    pickle.dump(tfidf_tran, handle)

# Load the model! 
t = pickle.load(open('/Users/ronak.k.bhatia/Desktop/Data_STA/tfid.pkl','rb'))

Now, I have to actually create a text file that contains all the words that I'm interested in.

In [ ]:
# !ls

In [ ]:
# !touch twitch_chat_analysis.txt

In [ ]:
# Opens the created text file and writes the vocab list to it! 
f=open('twitch_chat_analysis.txt','w')
s1='\n'.join(vocabulary)
f.write(s1)
f.close()

In [ ]:
# Saving the vocabulary
with open("/Users/ronak.k.bhatia/Desktop/topic-modeling-coursera-notes/twitch_chat_analysis.txt", "w") as file:
    file.write(str(vocabulary))

### Loading the vocabulary 
with open("/Users/ronak.k.bhatia/Desktop/topic-modeling-coursera-notes/twitch_chat_analysis.txt", "r") as file:
    data2 = eval(file.readline())


In [ ]:
# Loading the Google Universal Sentence Encoder's pretrained Model 

base_dir = "/Users/ronak.k.bhatia/Desktop/Data_STA/"

# !mkdir /Users/ronak.k.bhatia/Desktop/Data_STA/GoogleUSE Model
# !curl -L -o 4.tar.gz "https://tfhub.dev/google/universal-sentence-encoder/4?tf-hub-format=compressed" 
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"

# module_path ="/Users/ronak.k.bhatia/Desktop/Data_STA/GoogleUSE Model/USE_4"

%time model = hub.load(module_url)
#print ("module %s loaded" % module_url)

#Create function for using modeltraining
def embed(input):
    return model(input)


In [ ]:
# Running Training Process on Google USE Model 
ls =[]
chunksize =1000
le =len(df.message)
for i in range(0,le,chunksize):
    if(i+chunksize > le): 
        chunksize= le;
        ls.append(chunksize)
    else:
        a =i+chunksize
        ls.append(a)
ls
j=0

print(os)

for i in ls:
    directory = "/Users/ronak.k.bhatia/Desktop/topic-modeling-coursera-notes/Twitch-Data/" + str(i)
    if not os.path.exists(directory):
        os.makedirs(directory)
    directory = "/Users/ronak.k.bhatia/Desktop/topic-modeling-coursera-notes/Twitch-Data/" + str(i)
    print(j,i) 
    m=embed(df.message[j:i])
    exported_m = tf.train.Checkpoint(v=tf.Variable(m))
    exported_m.f = tf.function(
    lambda  x: exported_m.v * x,
    input_signature=[tf.TensorSpec(shape=None, dtype=tf.float32)])

    tf.saved_model.save(exported_m,directory)
    j = i
    print(i)

In [ ]:
# Batch-wise, loading the model! 
for i in ls:
    directory = "/Users/ronak.k.bhatia/Desktop/topic-modeling-coursera-notes/Twitch-Data/" + str(i)
    if os.path.exists(directory):
        print(directory)
        imported_m = tf.saved_model.load(directory)
        a= imported_m.v.numpy()
        #print(a)
        exec(f'load{i} = a')

In [ ]:
# Concatenate the array from the batchwise loaded model. 
con_a =np.concatenate((load1000, 
                       load2000,
                       load3000,
                       load4000,
                       load5000,
                       load6000,
                       load7000,
                       load8000,
                       load9000,
                       load10000,
                       load11000,
                       load12000,
                       load13000,
                       load14000,
                       load15000,
                       load16000,
                       load17000,
                       load18000,
                       load19000,
                       load20000,
                       load21000,
                       load22000,
                       load23000,
                       load24000,
                       load25000,
                       load25617,
                      ))
con_a.shape # makes sense because USE encoder outputs a 512 layer. 

In [ ]:
#Train the model
Model_USE= embed(df.message[0:3000])

In [ ]:
#Save the model
exported = tf.train.Checkpoint(v=tf.Variable(Model_USE))
exported.f = tf.function(
    lambda  x: exported.v * x,
    input_signature=[tf.TensorSpec(shape=None, dtype=tf.float32)])

tf.saved_model.save(exported,"/Users/ronak.k.bhatia/Desktop/topic-modeling-coursera-notes/Twitch-Data/")

In [235]:
#  Create function to get the top most similar documents by giving input keyword/sentence

def SearchDocument(query):
    q =[query]
    # embed the query for calcluating the similarity
    Q_Train =embed(q)
    
    #loadedmodel =imported_m.v.numpy()
    # Calculate the Similarity
    linear_similarities = linear_kernel(Q_Train, con_a).flatten() 
    #Sort top 10 index with similarity score
    Top_index_doc = linear_similarities.argsort()[:-400:-1]
    # sort by similarity score
    linear_similarities.sort()
    a = pd.DataFrame()
    for i,index in enumerate(Top_index_doc):
        a.loc[i,'index'] = str(index)
        a.loc[i,'Message in Chat'] = df['message'][index] ## Read File name with index from File_data DF
    for j,simScore in enumerate(linear_similarities[:-400:-1]):
        a.loc[j,'Score'] = simScore
    return a

In [241]:
SearchDocument('jacket')

,index,Message in Chat,Score
0,23915,!subscribercount,0.477913
1,23686,@shoootermcgavin 2xl,0.473387
2,2505,MR.UNLIMITED SHIRT,0.462875
3,21218,PauseChamping,0.459622
4,3928,loudly_crying_face:,0.458672
...,...,...,...
394,370,PauseChamp,0.391119
395,3176,PauseChamp,0.391119
396,21342,PauseChamp,0.391119
397,21199,PauseChamp,0.391119
